In [ ]:
%pip install ipywidgets
# %pip install solara

In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Artist selection widget
artist_toggle_options = ['At most k songs', 'At least k songs', 'None of', '--']
artist_toggle = widgets.Dropdown(
    options=artist_toggle_options,
    description='Preferences:',
    disabled=False,
    value='--'
)

# Number of songs for artist
artist_song_count = widgets.IntSlider(
    value=1,
    min=0,
    max=100,  # Adjust maximum as needed
    step=1,
    description='Songs Count:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='25%'),
    style={'description_width': 'initial'}
)

artist_name = widgets.Text(description='Artist name:')
song_name = widgets.Text(description='Song name:')

# artist_and_count = widgets.HBox([artist_toggle])
# artist_and_count.layout.align_items = 'center'

# intialize the output - second widget
out = Output()

def changed(change):
    '''
    Monitor change in the first widget
    '''
    global out

    if artist_toggle.value == 'At least k songs':
        out.append_display_data(artist_song_count)
#         display(out)
    elif artist_toggle.value == 'At most k songs':
        out.append_display_data(artist_song_count)
#         display(out)
    elif artist_toggle.value == 'None of':
        out.append_display_data(artist_name)
#         display(out)
    else:
        out.clear_output()
        out = Output()
    
    display(out)
    out.clear_output()   

                
# monitor the friends widget for changes
artist_toggle.observe(changed, 'value')

playlist_length_toggle = widgets.ToggleButtons(
    description='Playlist length:',
    options= ['Minutes', 'Number of Songs'])

# # Playlist length in minutes
# playlist_length_mins = widgets.IntRangeSlider(
#     value=[0, 120],
#     min=0,
#     max=120,  # 2 hours in minutes
#     step=10,  # step in minutes
#     description='Playlist Length (min):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # Playlist length in number of songs
# playlist_length_num_songs = widgets.IntSlider(
#     value=10,
#     min=2,
#     max=100,
#     step=1,
#     description='Playlist Length (number of songs):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # intialize the output - second widget
# out = Output()

# def changed(change):
#     '''
#     Monitor change in the first widget
#     '''
#     global out
#     if playlist_length_toggle.value == 'Minutes':  
#         out.append_display_data(playlist_length_mins)
#         # display(out)
#     elif playlist_length_toggle.value == 'Number of Songs':
#         out.append_display_data(playlist_length_num_songs)
#         # display(out)
#     else:
#         out.clear_output()
#         out = Output()
    
#     display(out)
#     out.clear_output()    
                
# monitor the friends widget for changes
playlist_length_toggle.observe(changed, 'value')


# BPM range widget
bpm_range = widgets.IntRangeSlider(
    value=[60, 180],  # Default BPM range
    min=0,
    max=300,
    step=1,
    description='BPM Range:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Loudness widget
loudness_range = widgets.FloatRangeSlider(
    value=[-60.0, 0.0],
    min=-60.0,
    max=0.0,
    step=0.1,
    description='Loudness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Liveness widget
liveness_range = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Liveness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

# Mode widget
mode_options = ['Major', 'Minor', 'Both']
mode_selection = widgets.Dropdown(
    options=mode_options,
    description='Mode:',
    disabled=False,
)

# Display widgets
display(artist_toggle, bpm_range, loudness_range, liveness_range, mode_selection)

Dropdown(description='Preferences:', index=3, options=('At most k songs', 'At least k songs', 'None of', '--')…

IntRangeSlider(value=(60, 180), continuous_update=False, description='BPM Range:', max=300)

FloatRangeSlider(value=(-60.0, 0.0), continuous_update=False, description='Loudness:', max=0.0, min=-60.0, rea…

FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='Liveness:', max=1.0, step=0.01)

Dropdown(description='Mode:', options=('Major', 'Minor', 'Both'), value='Major')

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "IntSlider(value=1, continuous_update=F…

In [32]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('data/spotify_playlists.tsv', sep='\t', index_col=0)
# df = df.sample(100)
# assign a random similarity score
df["similarity"] = np.random.rand(len(df))
df["duration_minutes"] = df['duration_ms'] / 60000
df["mode"]

0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
        ... 
15929    1.0
15930    1.0
15931    0.0
15932    1.0
15933    1.0
Name: mode, Length: 15934, dtype: float64

In [48]:
df.columns

Index(['artist_name', 'track_name', 'uri', 'num_plays', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'similarity', 'duration_minutes'],
      dtype='object')

In [49]:
bpm_range.value

(0, 300)

In [76]:
def apply_all_constraints(df):
    mode = {"Major": 1, "Minor" : 0}
    df = df.sort_values("similarity", ascending=False)
    df = df[(df["tempo"] > bpm_range.value[0]) & (df["tempo"] < bpm_range.value[1])]
    df = df[(df["loudness"] > loudness_range.value[0]) & (df["loudness"] < loudness_range.value[1])]
    df = df[(df["liveness"] > liveness_range.value[0]) & (df["liveness"] < liveness_range.value[1])]
    df = df[df["mode"] == mode[mode_selection.value]]
    import pandas as pd

    def apply_constraints(dataframe, playlist_length, artist_name, constraint_option, k):
        selected_rows = pd.DataFrame(columns=dataframe.columns)
        current_sum = 0
        artist_count = 0

        if constraint_option == 'at least k':
            # Add top rows corresponding to the artist until reaching at least k
            artist_rows = dataframe[dataframe['artist_name'] == artist_name].head(k)
            selected_rows = pd.concat([selected_rows, artist_rows])
            current_sum = selected_rows['duration_minutes'].sum()
            artist_count = len(artist_rows)

        for index, row in dataframe.iterrows():
            if index not in selected_rows.index:
                
                if current_sum + row['duration_minutes'] <= playlist_length + 10:
                    # Check constraint 2: At most k, at least k, or none songs from the specified artist
                    if constraint_option == 'at most k' and artist_count < k:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']
                        if artist_name == row['artist_name']:
                            artist_count += 1
                    elif constraint_option == 'none' and artist_name != row['artist_name'] and artist_count == 0:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']
                    else:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']

                # Check constraint 1: Sum of column_1 within +/- 10 of the target_sum
                if current_sum >= playlist_length - 10 and current_sum <= playlist_length + 10:
                    break

        return selected_rows

    return apply_constraints(df, playlist_length=150, artist_name='Taylor Swift', constraint_option='at least k', k=4)
    # TODOOOOO: change the above arguments to the values from the togles ^

apply_all_constraints(df)

/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/4001595999.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/4001595999.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/4001595999.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/4001595999.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  se

,artist_name,track_name,uri,num_plays,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,similarity,duration_minutes
7729,Taylor Swift,Red,spotify:track:0cITLOYn1Sv4q27zZPqlNK,2,0.602,0.896,1.0,-4.267,0.0,0.0437,...,0.641,124.978,audio_features,0cITLOYn1Sv4q27zZPqlNK,https://api.spotify.com/v1/tracks/0cITLOYn1Sv4...,https://api.spotify.com/v1/audio-analysis/0cIT...,220827.0,4.0,0.870905,3.680450
2136,Taylor Swift,I Knew You Were Trouble.,spotify:track:6FB3v4YcR57y4tXFcdxI1E,29,0.622,0.469,3.0,-6.798,0.0,0.0363,...,0.679,77.019,audio_features,6FB3v4YcR57y4tXFcdxI1E,https://api.spotify.com/v1/tracks/6FB3v4YcR57y...,https://api.spotify.com/v1/audio-analysis/6FB3...,219720.0,4.0,0.869061,3.662000
9213,Taylor Swift,Only The Young - Featured in Miss Americana,spotify:track:2slqvGLwzZZYsT4K4Y1GBC,1,0.560,0.780,5.0,-4.895,0.0,0.0517,...,0.602,93.516,audio_features,2slqvGLwzZZYsT4K4Y1GBC,https://api.spotify.com/v1/tracks/2slqvGLwzZZY...,https://api.spotify.com/v1/audio-analysis/2slq...,157507.0,4.0,0.651194,2.625117
7739,Taylor Swift,Better Than Revenge,spotify:track:1By2VQkUN0Frd8mKNV5NYC,2,0.516,0.917,11.0,-3.186,0.0,0.0827,...,0.635,145.821,audio_features,1By2VQkUN0Frd8mKNV5NYC,https://api.spotify.com/v1/tracks/1By2VQkUN0Fr...,https://api.spotify.com/v1/audio-analysis/1By2...,217160.0,4.0,0.540486,3.619333
4081,Lil Uzi Vert,Leaders (feat. NAV),spotify:track:4D7NrSeqkTarBrJ80b2sBc,7,0.950,0.654,0.0,-5.895,0.0,0.3070,...,0.886,132.936,audio_features,4D7NrSeqkTarBrJ80b2sBc,https://api.spotify.com/v1/tracks/4D7NrSeqkTar...,https://api.spotify.com/v1/audio-analysis/4D7N...,194844.0,4.0,0.999867,3.247400
14604,SamuW,Jiyu,spotify:track:07Sx03dUY4jRXXtrM7zSs3,1,0.783,0.183,0.0,-10.173,0.0,0.1030,...,0.287,140.049,audio_features,07Sx03dUY4jRXXtrM7zSs3,https://api.spotify.com/v1/tracks/07Sx03dUY4jR...,https://api.spotify.com/v1/audio-analysis/07Sx...,138706.0,4.0,0.999819,2.311767
7797,The Chordettes,Lollipop,spotify:track:1hFB1obL3dCyIDigkojSTm,2,0.650,0.593,7.0,-6.374,0.0,0.1220,...,0.825,143.179,audio_features,1hFB1obL3dCyIDigkojSTm,https://api.spotify.com/v1/tracks/1hFB1obL3dCy...,https://api.spotify.com/v1/audio-analysis/1hFB...,130533.0,4.0,0.999815,2.175550
11725,dvsn,What's Up (feat. Jagged Edge),spotify:track:2HI6OFcMNUyS0IEwp4hffV,1,0.657,0.655,10.0,-4.048,0.0,0.0465,...,0.651,132.153,audio_features,2HI6OFcMNUyS0IEwp4hffV,https://api.spotify.com/v1/tracks/2HI6OFcMNUyS...,https://api.spotify.com/v1/audio-analysis/2HI6...,219729.0,4.0,0.999768,3.662150
8887,miaww,Escape You,spotify:track:4j7UbW1si5RwWBi08nMrWm,1,0.612,0.423,6.0,-9.238,0.0,0.0303,...,0.101,119.966,audio_features,4j7UbW1si5RwWBi08nMrWm,https://api.spotify.com/v1/tracks/4j7UbW1si5Rw...,https://api.spotify.com/v1/audio-analysis/4j7U...,202026.0,4.0,0.999583,3.367100
1314,Tiffany Day,FEEL ALRIGHT,spotify:track:1E7CteauN9JfKayLvs5rkk,47,0.675,0.553,7.0,-8.484,0.0,0.1050,...,0.243,77.490,audio_features,1E7CteauN9JfKayLvs5rkk,https://api.spotify.com/v1/tracks/1E7CteauN9Jf...,https://api.spotify.com/v1/audio-analysis/1E7C...,221921.0,4.0,0.999537,3.698683


In [69]:
df["tempo"].describe()

count    6443.000000
mean      118.243458
std        28.825456
min        38.695000
25%        95.025000
50%       117.983000
75%       137.086000
max       219.378000
Name: tempo, dtype: float64

In [70]:
filtering_criterion = {
    "Author": "SZA",
    "ms_played": lambda x: x < 100_000
}

/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/3509761256.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/3509761256.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/3509761256.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_rows = selected_rows.append(row)
/var/folders/n8/y__hwc457_z0wx535z5l8lgr0000gn/T/ipykernel_68686/3509761256.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  se

,artist_name,track_name,uri,num_plays,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,similarity,duration_minutes
7729,Taylor Swift,Red,spotify:track:0cITLOYn1Sv4q27zZPqlNK,2,0.602,0.896,1.0,-4.267,0.0,0.0437,...,0.641,124.978,audio_features,0cITLOYn1Sv4q27zZPqlNK,https://api.spotify.com/v1/tracks/0cITLOYn1Sv4...,https://api.spotify.com/v1/audio-analysis/0cIT...,220827.0,4.0,0.870905,3.680450
2136,Taylor Swift,I Knew You Were Trouble.,spotify:track:6FB3v4YcR57y4tXFcdxI1E,29,0.622,0.469,3.0,-6.798,0.0,0.0363,...,0.679,77.019,audio_features,6FB3v4YcR57y4tXFcdxI1E,https://api.spotify.com/v1/tracks/6FB3v4YcR57y...,https://api.spotify.com/v1/audio-analysis/6FB3...,219720.0,4.0,0.869061,3.662000
9213,Taylor Swift,Only The Young - Featured in Miss Americana,spotify:track:2slqvGLwzZZYsT4K4Y1GBC,1,0.560,0.780,5.0,-4.895,0.0,0.0517,...,0.602,93.516,audio_features,2slqvGLwzZZYsT4K4Y1GBC,https://api.spotify.com/v1/tracks/2slqvGLwzZZY...,https://api.spotify.com/v1/audio-analysis/2slq...,157507.0,4.0,0.651194,2.625117
7739,Taylor Swift,Better Than Revenge,spotify:track:1By2VQkUN0Frd8mKNV5NYC,2,0.516,0.917,11.0,-3.186,0.0,0.0827,...,0.635,145.821,audio_features,1By2VQkUN0Frd8mKNV5NYC,https://api.spotify.com/v1/tracks/1By2VQkUN0Fr...,https://api.spotify.com/v1/audio-analysis/1By2...,217160.0,4.0,0.540486,3.619333
4081,Lil Uzi Vert,Leaders (feat. NAV),spotify:track:4D7NrSeqkTarBrJ80b2sBc,7,0.950,0.654,0.0,-5.895,0.0,0.3070,...,0.886,132.936,audio_features,4D7NrSeqkTarBrJ80b2sBc,https://api.spotify.com/v1/tracks/4D7NrSeqkTar...,https://api.spotify.com/v1/audio-analysis/4D7N...,194844.0,4.0,0.999867,3.247400
14604,SamuW,Jiyu,spotify:track:07Sx03dUY4jRXXtrM7zSs3,1,0.783,0.183,0.0,-10.173,0.0,0.1030,...,0.287,140.049,audio_features,07Sx03dUY4jRXXtrM7zSs3,https://api.spotify.com/v1/tracks/07Sx03dUY4jR...,https://api.spotify.com/v1/audio-analysis/07Sx...,138706.0,4.0,0.999819,2.311767
7797,The Chordettes,Lollipop,spotify:track:1hFB1obL3dCyIDigkojSTm,2,0.650,0.593,7.0,-6.374,0.0,0.1220,...,0.825,143.179,audio_features,1hFB1obL3dCyIDigkojSTm,https://api.spotify.com/v1/tracks/1hFB1obL3dCy...,https://api.spotify.com/v1/audio-analysis/1hFB...,130533.0,4.0,0.999815,2.175550
11725,dvsn,What's Up (feat. Jagged Edge),spotify:track:2HI6OFcMNUyS0IEwp4hffV,1,0.657,0.655,10.0,-4.048,0.0,0.0465,...,0.651,132.153,audio_features,2HI6OFcMNUyS0IEwp4hffV,https://api.spotify.com/v1/tracks/2HI6OFcMNUyS...,https://api.spotify.com/v1/audio-analysis/2HI6...,219729.0,4.0,0.999768,3.662150
8887,miaww,Escape You,spotify:track:4j7UbW1si5RwWBi08nMrWm,1,0.612,0.423,6.0,-9.238,0.0,0.0303,...,0.101,119.966,audio_features,4j7UbW1si5RwWBi08nMrWm,https://api.spotify.com/v1/tracks/4j7UbW1si5Rw...,https://api.spotify.com/v1/audio-analysis/4j7U...,202026.0,4.0,0.999583,3.367100
1314,Tiffany Day,FEEL ALRIGHT,spotify:track:1E7CteauN9JfKayLvs5rkk,47,0.675,0.553,7.0,-8.484,0.0,0.1050,...,0.243,77.490,audio_features,1E7CteauN9JfKayLvs5rkk,https://api.spotify.com/v1/tracks/1E7CteauN9Jf...,https://api.spotify.com/v1/audio-analysis/1E7C...,221921.0,4.0,0.999537,3.698683


In [ ]:
%pip install python-constraint

In [ ]:
# FROM JESS'S Inference.ipynb

# function to get top-n most similar tracks
def get_most_similar_tracks(track_name, n=10, tokenizer=tokenizer, embedding_weights=embedding_weights):
    
    # get track embedding
    track_idx = tokenizer.word_index[track_name]
    track_vector = embedding_weights[track_idx, :].reshape(1, -1)

    # compute similarities against other tracks
    similarities = np.dot(track_vector, embedding_weights.T) / (np.linalg.norm(track_vector) * np.linalg.norm(embedding_weights, axis=1))
    similarities = similarities.reshape(-1)

    # get most similar tracks' indices
    most_similar_idxs = np.argpartition(similarities, -(n+1))[-(n+1):]
    most_similar_idxs = most_similar_idxs[np.argsort(similarities[most_similar_idxs])][::-1][1:]

    # print most similar tracks, along with their positions in training data
    print("top {} tracks most similar to '{}' (pos. {}):".format(n, track_name, track_idx))
    for idx in most_similar_idxs:
        print("- (sim. {:.3f}): '{}' (pos. {})".format(similarities[idx], tokenizer.index_word[idx], idx))